The PACS maps for HELP have been made by Matt Smith and Kyle Penner. Unfortunetly, the formats differ between the two. For instance Kyle's noise maps are variance rather than standard deviation maps and units differ. This script creates an homogenised format, and attaches other relevant meta data and the ObsIDs.

Steps:
for each field:
* read in coverage, image and noise
* if Kyle's convert units and correct noise
* attach obs IDs
* attach meta data
* create PSF extension

In [1]:
from datetime import datetime
from itertools import product
import glob

import numpy as np

from astropy.io import fits
from astropy.table import Table
VERSION = "0.9"

In [2]:
filelist= glob.glob("/Volumes/Maxtor/Data/mips_fields/*/*1.mosaic.fits", recursive=True)

In [3]:
print(filelist)

['/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/60096211.60096211-8.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/60096211.60096211-9.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40006880.40006880-0.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40007720.40007720-0.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40016180.40016180-0.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40016410.40016410-0.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40019880.40019880-0.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40024860.40024860-0.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/60090290.60090290-0.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/60096211.60096211-1.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/60096211.60096211-6.MIPS.1.mosaic.fits', '/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40039070.40039070-0.

In [ ]:
for i, file in enumerate(filelist):
    
    print(file)
    folders=file.split('/')
    print,folders[4]
    field=folders[4]
 #   stop
    ext=0
    
    try:
       # Load in image map
        image_map=fits.open(file)
        image_hdu = fits.ImageHDU(header=image_map[ext].header,
                                data=image_map[ext].data)
        image_hdu.header['EXTNAME'] = "IMAGE"
 
    # Load in uncertainty map
        noise_map=fits.open(file.replace('mosaic','unc'))
        noise_hdu = fits.ImageHDU(header=noise_map[ext].header,
                                data=noise_map[ext].data)
        noise_hdu.header['EXTNAME'] = "ERROR"
 
    # Load in coverage map
        cov_map=fits.open(file.replace('mosaic','cov'))
        cov_hdu = fits.ImageHDU(header=cov_map[ext].header,
                                data=cov_map[ext].data)
        cov_hdu.header['EXTNAME'] = "COVERAGE"

        
    except:
        #fits.close(file)
        pass
            
    #Create primary header
    primary_hdu = fits.PrimaryHDU()
    primary_hdu.header.append((
            "CREATOR", "Herschel Extragalactic Legacy Project"
        ))
    primary_hdu.header.append((
            "TIMESYS", "UTC", "All dates are in UTC time"
        ))
    primary_hdu.header.append((
            "DATE", datetime.now().replace(microsecond=0).isoformat(),
            "Date of file creation"
    ))
    primary_hdu.header.append((
        "VERSION", VERSION, "HELP product version"
    ))
    primary_hdu.header.append((
        "TELESCOP", "Spitzer", "Name of the telescope"
    ))
    primary_hdu.header.append((
        "INSTRUME", "MIPS", "Name of the instrument"
    ))
    primary_hdu.header.append((
        "FILTER", "MIPS-24", "Name of the filter"
    ))
    primary_hdu.header.append((
        "FIELD", field, "Name of the HELP field"
    ))
#    for idx, obs_id in enumerate(obsids):
#        keyword = "OBSID" + str(idx).zfill(3)
#        primary_hdu.header.append((keyword, obs_id))


    #Add empty extension for PSF
    psf_hdu = fits.ImageHDU()
    psf_hdu.header['EXTNAME'] = "PSF"
    psf_hdu.header.add_comment("The PSF is not available.")

    hdu_list = fits.HDUList([primary_hdu, image_hdu, noise_hdu, cov_hdu, psf_hdu])
    hdu_list.writeto(file.replace('mosaic','help'),
                 checksum=True, overwrite=True)
    print(f"{file} /  processed...")

/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/60096211.60096211-8.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/60096211.60096211-8.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/60096211.60096211-9.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/60096211.60096211-9.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40006880.40006880-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40006880.40006880-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40007720.40007720-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40007720.40007720-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40016180.40016180-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40016180.40016180-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/AKARI-NEP/40016410.40016410-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Da

/Volumes/Maxtor/Data/mips_fields/Bootes/40039460.40039460-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Bootes/50061270.50061270-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Bootes/50061270.50061270-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Bootes/50062840.50062840-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Bootes/50062840.50062840-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Bootes/60087831.60087831-5.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Bootes/60087831.60087831-5.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Bootes/60087831.60087831-22.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Bootes/60087831.60087831-22.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Bootes/60087831.60087831-33.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Bootes/60087831.60087831-33.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_field

/Volumes/Maxtor/Data/mips_fields/CDFS-SWIRE/60095881.60095881-4.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/CDFS-SWIRE/60095881.60095881-5.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/CDFS-SWIRE/60095881.60095881-5.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/CDFS-SWIRE/60095881.60095881-11.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/CDFS-SWIRE/60095881.60095881-11.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/COSMOS/60095931.60095931-9.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/COSMOS/60095931.60095931-9.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/COSMOS/40025170.40025170-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/COSMOS/40025170.40025170-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/COSMOS/40030190.40030190-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/COSMOS/40030190.40030190-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxt

/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/40033850.40033850-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/40034940.40034940-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/40034940.40034940-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/50062660.50062660-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/50062660.50062660-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/60079760.60079760-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/60079760.60079760-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/60086021.60086021-33.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/60086021.60086021-33.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/60084300.60084300-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/ELAIS-N2/60084300.60084300-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Ma

/Volumes/Maxtor/Data/mips_fields/GAMA-09/40041100.40041100-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-09/40042380.40042380-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-09/40042380.40042380-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-09/40043770.40043770-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-09/40043770.40043770-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-09/50066370.50066370-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-09/50066370.50066370-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-09/50066600.50066600-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-09/50066600.50066600-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-09/50067260.50067260-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-09/50067260.50067260-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mip

/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000841.20000841-363.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000781.20000781-181.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000781.20000781-181.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000811.20000811-131.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000811.20000811-131.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000811.20000811-363.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000811.20000811-363.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000811.20000811-545.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000811.20000811-545.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000841.20000841-131.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-12/20000841.20000841-131.MIPS.1.mosaic.fits /  processed...
/V

/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-22.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-3.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-3.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-4.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-4.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-5.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-5.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-6.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-6.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-1.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/GAMA-15/60096291.60096291-1.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mi

/Volumes/Maxtor/Data/mips_fields/Herschel-Stripe-82/20000240.20000240-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Herschel-Stripe-82/20000271.20000271-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Herschel-Stripe-82/20000271.20000271-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Herschel-Stripe-82/20000271.20000271-1.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Herschel-Stripe-82/20000271.20000271-1.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Herschel-Stripe-82/20000271.20000271-2.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Herschel-Stripe-82/20000271.20000271-2.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Herschel-Stripe-82/20000271.20000271-4.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Herschel-Stripe-82/20000271.20000271-4.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Herschel-Stripe-82/20000271.20000271-5.MIPS.1.mosaic.fits
/Volumes/M

/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/40024550.40024550-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/60094440.60094440-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/60094440.60094440-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/60095761.60095761-33.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/60095761.60095761-33.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/40028050.40028050-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/40028050.40028050-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/40031590.40031590-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/40031590.40031590-0.MIPS.1.mosaic.fits /  processed...
/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/40033350.40033350-0.MIPS.1.mosaic.fits
/Volumes/Maxtor/Data/mips_fields/Lockman-SWIRE/40033350.40

In [ ]:
print,(noise_hdu.header)